In [25]:
import pandas as pd 
import import_ipynb
import DataSetManipulation
import TrainModel
import joblib
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [26]:

cleanFile = 'ImplementationLogs.csv'
finalFile = 'ImplementationTestLogs.csv'

minMessages = 100
maxMessages = 10000
messagesPerColumn = 30

#Edits extracted logs csv file to be compatible with existing program
df = pd.read_csv('RealChatLogs.csv')

df = df[['Username', 'Content']]
df.rename(columns={
     'Username': 'User',
     'Content': 'Message'
 }, inplace = True)

df.to_csv('ImplementationLogs.csv', index = False)

print("Edits to fit the correct format done.")

#Filters messages so that amount of variables changing when going for implementation stay low.
data, keepList = DataSetManipulation.MinMaxFilter(cleanFile, minMessages, maxMessages)
print('Filtering by min and max messages done.')

#writes the final csv file that can be used to test the current model.
DataSetManipulation.PairData(data, keepList, finalFile, messagesPerColumn)
print('Final dataset done.')


Edits to fit the correct format done.
Filtering by min and max messages done.


Processing users: 100%|██████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 24.04it/s]

Final dataset done.


In [27]:
#creates the message to vector dictionary nessesary for cosine comparison
df, messageToVectorDictionary = TrainModel.MessageToVectorDictionaryBuilder(finalFile)

#Builds features for testing
df = TrainModel.FeatureBuilder(df, messageToVectorDictionary)

In [29]:
#loads the model and runs the tests on the prediction 
model = joblib.load("RandomForestModel.pkl")

X = df.drop(columns=[
    'Message1',
    'Message2', 
    'FromSameUserNum',
    'FromSameUser'
])
true = df["FromSameUserNum"]

# Make predictions
prediction = model.predict(X)

print(classification_report(true, prediction))

print("Confusion Matrix:")
print(confusion_matrix(true, prediction))

              precision    recall  f1-score   support

           0       1.00      0.29      0.44         7
           1       0.58      1.00      0.74         7

    accuracy                           0.64        14
   macro avg       0.79      0.64      0.59        14
weighted avg       0.79      0.64      0.59        14

Confusion Matrix:
[[2 5]
 [0 7]]
